In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows

# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Ideal_Target_Data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['Ideal_Target_Updated_Target_ESDT_ES 09-24_Minute_60.csv',
 'Ideal_Target_Updated_Target_ESDT_NQ 09-24_Minute_60.csv',
 'Ideal_Target_Updated_Target_ESDT_ZB 09-24_Minute_60.csv']

In [16]:
s = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Ideal_Target_Data/Ideal_Target_Updated_Target_ESDT_ES 09-24_Minute_60.csv")

In [17]:
s.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice', 'Entry', 'Defence',
       'Stop', 'ideal_target'],
      dtype='object')

In [18]:
import os
import pandas as pd
from tqdm import tqdm

def process_data(df):
    # Initialize columns with 0
    df['Buyer_Profit'] = 0
    df['Buyer_Loss'] = 0
    df['Seller_Profit'] = 0
    df['Seller_Loss'] = 0

    # Convert 'Time Stamp' to datetime for efficient comparison
    df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])

    # Use a list to collect results for better performance
    results = []

    # Iterate through each row to perform the logic
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Processing Rows'):
        # Calculate Buyer Profit and Loss
        if row['BuyersInControlPrice'] != 0:
            entry_value = row['Entry']
            target_value = row['Target']

            if target_value > entry_value:
                buyer_profit = target_value - entry_value
                buyer_loss = 0
            else:
                buyer_profit = 0
                buyer_loss = abs(target_value - entry_value)
        else:
            buyer_profit = 0
            buyer_loss = 0

        # Calculate Seller Profit and Loss
        if row['SellersInControlPrice'] != 0:
            entry_value = row['Entry']
            target_value = row['Target']

            if entry_value > target_value:
                seller_profit = entry_value - target_value
                seller_loss = 0
            else:
                seller_profit = 0
                seller_loss = abs(entry_value - target_value)
        else:
            seller_profit = 0
            seller_loss = 0

        # Append results
        results.append({
            'index': index,
            'Buyer_Profit': buyer_profit,
            'Buyer_Loss': buyer_loss,
            'Seller_Profit': seller_profit,
            'Seller_Loss': seller_loss
        })

    # Update DataFrame with results
    results_df = pd.DataFrame(results)
    df.update(results_df.set_index('index'))
    df['Profit'] = df['Buyer_Profit'] + df['Seller_Profit']
    df['Loss'] = df['Buyer_Loss'] + df['Seller_Loss']
    df.drop(columns=["Buyer_Profit", "Buyer_Loss", "Seller_Profit", "Seller_Loss"], axis=1, inplace=True)

    return df

def process_files_in_directory(input_directory, output_directory):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Get all CSV files in the input directory
    files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]

    for file in files:
        file_path = os.path.join(input_directory, file)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)


        # # Rename the 'ideal_target' column to 'Target'
        # df.drop(columns=["ED_Target"],axis=1,inplace=True)
        if 'ideal_target' in df.columns:
            df.rename(columns={'ideal_target': 'Target'}, inplace=True)

        # df.drop(columns=["ED_Target"],axis=1,inplace=True)

        # Process the DataFrame
        df = process_data(df)

        # Save the modified DataFrame to the output directory with a new file name
        output_file_name = f"Processed_{file}"
        output_file_path = os.path.join(output_directory, output_file_name)
        df.to_csv(output_file_path, index=False)

        print(f"Processed and saved {output_file_name} to {output_directory}")

In [19]:
# Example usage
output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Profit_Loss_Data/Profit_Loss_Ideal_Data'
process_files_in_directory(data_folder, output_directory)

Processing Rows: 100%|██████████| 43580/43580 [00:03<00:00, 13635.66it/s]


Processed and saved Processed_Ideal_Target_Updated_Target_ESDT_ES 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Profit_Loss_Data/Profit_Loss_Ideal_Data


Processing Rows: 100%|██████████| 44747/44747 [00:04<00:00, 9270.47it/s] 


Processed and saved Processed_Ideal_Target_Updated_Target_ESDT_NQ 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Profit_Loss_Data/Profit_Loss_Ideal_Data


Processing Rows: 100%|██████████| 41532/41532 [00:03<00:00, 13626.18it/s]


Processed and saved Processed_Ideal_Target_Updated_Target_ESDT_ZB 09-24_Minute_60.csv to /content/drive/MyDrive/Real_Trader_EDA_R1/60_Minute_Data/Calculate_60_Minute_Data/Profit_Loss_Data/Profit_Loss_Ideal_Data
